<!-- Auto table of contents -->
<h1 class='tocIgnore'>Naturalist : Extraction et exploitation des traces</h1>
<ul>
  <li>présentes dans les formulaires à partir Naturalist V0.128 (ou beta mai 2019),</li>
  <li>à condition de cocher la case "Enregistrer ma trace" en début de formulaire,</li>
  <li>via l'export Excel exclusivement
      (pas encore d'API pour ça, et absent des exports XML, JSON, KML, CSV en décembre 2019),</li>
  <li>avec la colonne "trace" sélectionnée dans l'export,</li>
  <li>uniquement via Faune-France (pas dispo. via les sites régionaux).</li>
</ul>
<p>Lecture XLSX et carto. publiée avec données anonymisées sur https://framagit.org/lpo/partage-de-codes le 25/01/2020</p>
<div style="overflow-y: auto">
  <h2 class='tocIgnore'>Table des matières</h2>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('ipython_notebook_toc.js')

In [ ]:
import sys
import os
import pathlib as pl
import datetime as dt
import pandas as pd
import numpy as np

import folium
import folium.plugins

from collections import OrderedDict as odict
import json

from IPython.display import HTML

# Communs

In [ ]:
# Serveurs de couche carto. pour folium / Leaflet
mdOSM = dict(tiles='http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
             attr='Open Street Map',
             name='Open Street Map', max_zoom=22)

mdOTM = dict(tiles='http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
             attr='<a href="https://opentopomap.org/">OpenTopoMap</a> '
                  '(<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)',
             name='Open Topo Map', max_zoom=22)
mdThOut = dict(tiles='https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png',
               attr='Thunderforest Outdoors', 
               name='Thunderforest Outdoors', max_zoom=22)

mdSatArcGis = dict(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                   attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid,'
                        ' IGN, IGP, UPR-EGP, and the GIS User Community',
                   name='ArcGIS Satellite',
                   max_zoom=22)

# Chargement des données (exports de Faune-France)

In [ ]:
# ACDC 2019 JPM (en fait, traceenregistrée uniquement sur le 2nd passage, et pour 15 points sur 17)
fn = f'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602ff.xlsx'
obser, obserAbbv = 'Jean-Philippe Meuret', 'JPM'

In [ ]:
# Formulaires Romain avec trace du 25/05/2019 au 25/01/2020 (tous)
# * Impluvium: "epoc" & Saint-Ours, Pulvérières, Charbonnière-les-Varennes
# * et le reste : des EPOC et des transects
fn = f'tmp/FormulairesRR-traces-20190525-20200125ff.xlsx'
obser, obserAbbv = 'Romain Riols', 'RR'

In [ ]:
# ACDC 2019 Romain 25/05/2019
fn = f'tmp/ACDC2019-Naturalist-FormulairesRR20190425ff.xlsx'
obser, obserAbbv = 'Romain Riols', 'RR'

In [ ]:
# Test JPM jardin 23/05/2019
#fn = f'tmp/NaturalistTestTrace-JardinJPM20190523ff.xlsx'
#obser, obserAbbv = 'Jean-Philippe Meuret', 'JPM'

In [ ]:
obsCols = ['ID liste', 'Liste complète ?', 'Commentaire de la liste',
           'Date', 'Horaire', 'Lieu-dit', 'Commune', 'Nom espèce',
           'Estimation', 'Nombre', 'Détails', 'Code atlas',
           'Lat (WGS84)', 'Lon (WGS84)',
           'Remarque', 'Remarque privée', 'Prénom', 'Nom',
           'Protocole', 'Trace']

In [ ]:
dfObs = pd.read_excel(fn)

# Uniquement les données des formulaires
dfObs.drop(dfObs[dfObs['ID liste'] == 0].index, inplace=True) 

dfObs[obsCols]

In [ ]:
# Nettoyage colonnes inutiles
dfObs.drop(columns=[col for col in dfObs.columns if col.startswith('SEARCH_EXPORT')], inplace=True)
dfObs.columns

# Filtrage des données

(on ne garde que celles des formulaires)

In [ ]:
dfObs['Commentaire de la liste'].unique()

In [ ]:
# Uniquement ceux d'ACDC.
dfObs.drop(dfObs[dfObs['Commentaire de la liste'].isnull()].index, inplace=True)
dfObs.drop(dfObs[~dfObs['Commentaire de la liste'].str.contains('ACDC', case=False)].index, inplace=True)

In [ ]:
# Uniquement ceux avec trace
dfObs.drop(dfObs[dfObs['Trace'].isnull()].index, inplace=True)

len(dfObs[obsCols])

# Examen des données

In [ ]:
# Normalement, 1 trace par formulaire
dfObs['Trace'].unique()

In [ ]:
dfObs[['ID liste', 'Trace']].groupby('ID liste').nunique()

In [ ]:
# Vérifier que la trace de la liste est présente à l'identique dans toutes les données de la liste
assert all(dfObs[['ID liste', 'Trace']].groupby('ID liste').nunique().Trace == 1)

In [ ]:
# Les formulaires à traiter
dfObs[['ID liste', 'Liste complète ?', 'Commentaire de la liste', 'Trace']].groupby('ID liste').first()

In [ ]:
# Les formulaires à traiter : nbre de données
dfObs[['ID liste', 'Liste complète ?', 'Commentaire de la liste', 'Trace']].groupby('ID liste').count().Trace

# Extraction et décodage des traces

In [ ]:
# On peut donc récupérer trace de chaque liste et en extraire les points individuels
formIndCols = ['ID liste', 'Date', 'Liste complète ?', 'Commentaire de la liste']
dfTraces = dfObs[formIndCols + ['Trace']].groupby(formIndCols).first()
dfTraces

In [ ]:
# Exemple de trace
dfTraces.iloc[0].Trace

In [ ]:
def decoderTrace(trace):
    return [[float(num) for num in xy.strip().split(' ')] for xy in trace[len('LINESTRING('):-len(')')].split(',')]
    
dfTraces.Trace = dfTraces.Trace.apply(decoderTrace)
dfTraces = dfTraces.Trace.apply(pd.Series).stack().reset_index()
dfTraces[['obseur_lon', 'obseur_lat']] = dfTraces.loc[:, 0].apply(pd.Series)
dfTraces.drop(columns=[0], inplace=True)
dfTraces.rename(columns=dict(level_4='NumPt'), inplace=True)
dfTraces

# Tracé cartographique des données d'un formulaire et de sa trace

In [ ]:
idListe = 766164

In [ ]:
dfTraceListe = dfTraces.loc[dfTraces['ID liste'] == idListe].copy()
dfTraceListe

In [ ]:
dfObsListe = dfObs.loc[dfObs['ID liste'] == idListe,
                       ['Lon (WGS84)', 'Lat (WGS84)', 'Date', 'Horaire', 'Nom espèce', 'Nombre', 'Code atlas']]
dfObsListe

In [ ]:
mapDesc = dict(tiles='http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
               attr='<a href="https://opentopomap.org/">OpenTopoMap</a> '
                    '(<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)',
               max_zoom=22) # OK
#mapDesc = dict(tiles='http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 
#               attr='osm',
#               max_zoom=22) # OK
#mapDesc = dict(tiles='https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png',
#               attr='thunderforest',
#               max_zoom=22) # OK

#mapDesc = dict(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#               attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid,'
#                    ' IGN, IGP, UPR-EGP, and the GIS User Community',
#               max_zoom=22)

mp = folium.Map(**mapDesc)

# La trace
# a. Le lignes reliant les points
dfTraceListe['obseur_lon_sfd'] = dfTraceListe.obseur_lon.shift(-1)
dfTraceListe['obseur_lat_sfd'] = dfTraceListe.obseur_lat.shift(-1)
dfTraceListe.dropna(inplace=True)

commListe = dfTraceListe.iloc[0]['Commentaire de la liste']
lines = list(zip(zip(dfTraceListe.obseur_lat, dfTraceListe.obseur_lon),
                 zip(dfTraceListe.obseur_lat_sfd, dfTraceListe.obseur_lon_sfd)))[:-1]
pline = folium.PolyLine(lines, color='blue', weight=1, opacity=0.6,
                        popup=folium.Popup('#{} {}'.format(idListe, commListe)))
pline.add_to(mp)

# b. Les points
for _, sPt in dfTraceListe.iterrows():
    mrk = folium.CircleMarker(location=(sPt.obseur_lat, sPt.obseur_lon), 
                              popup=folium.Popup('#{}: {}'.format(sPt['ID liste'], sPt.NumPt)),
                              radius=2, weight=2, color='red', fill=True)
    mrk.add_to(mp)

# Les données
for indObs, sObs in dfObsListe.iterrows():
    mrk = folium.CircleMarker(location=(sObs['Lat (WGS84)'], sObs['Lon (WGS84)']), 
                              radius=3, color='cyan', fill=True,
                              popup=folium.Popup('#{} {} {} {} {} (code {})' \
                                                 .format(indObs, sObs.Date, sObs.Horaire,
                                                         sObs.Nombre, sObs['Nom espèce'], sObs['Code atlas'])))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

# Comparaison traces / géolocs "réelles"

* entre mes formulaires ACDC 2019 2nd passage, quasi-tous "avec trace",
* et les géolocs "de mémoire" (novembre 2019).

In [ ]:
# Lecture des données brutes avec géolocs de mémoire (produites via NB ACDC-donnees-naturalist IV.4)
dfObsBrutes = pd.read_excel('ACDC/ACDC2019-Naturalist-ObsBrutesAvecDist.xlsx')
dfObsBrutes.drop(dfObsBrutes[dfObsBrutes.Observateur != obser].index, inplace=True)
dfObsBrutes[['Num point ACDC', 'Passage', 'Date', 'Heure début', 'lon_mem', 'lat_mem']]

In [ ]:
dfObsBrutes['Num point ACDC'].unique()

In [ ]:
# Extraction des géolocs de mémoire
dfGeolocMem = dfObsBrutes[['Num point ACDC', 'Passage', 'Date', 'Heure début', 'lon_mem', 'lat_mem']] \
                .groupby(['Num point ACDC', 'Passage']).first()
dfGeolocMem

In [ ]:
# Traces : Ajouts infos Num point ACDC et Passage
# (puisque ID liste spécifiques à Faune XX, différent de ceux de Faune France)
dfTraces['Num point ACDC'] = dfTraces['Commentaire de la liste'].apply(lambda s: int(s.split(' ')[1]))
dfTraces['Passage'] = dfTraces.Date.apply(lambda ts: 'a' if ts < pd.Timestamp('2019-05-15') else 'b')
dfTraces

In [ ]:
# Jointure avec les traces (en gardant tous les points et passages effectués, pas seulement ceux avec trace)
dfTraces = dfTraces.join(dfGeolocMem[['lon_mem', 'lat_mem']], on=['Num point ACDC', 'Passage'], how='right')
dfTraces.reset_index(inplace=True)
dfTraces

In [ ]:
# Suppression des traces du passage a (non dispo. à l'époque)
dfTraces.drop(dfTraces[dfTraces.Passage == 'a'].index, inplace=True)
dfTraces

In [ ]:
dfTraces['Num point ACDC'].unique(), dfTraces['Passage'].unique()

In [ ]:
# Cartographie des formulaires
mp = folium.Map()
folium.TileLayer(**mdSatArcGis).add_to(mp)

# Le contrôle pour changer de couche
folium.LayerControl().add_to(mp)

# Pour chaque point ...
for numPoint in sorted(dfTraces['Num point ACDC'].unique()):
    
    # Les données
    dfObsListe = dfObsBrutes[dfObsBrutes['Num point ACDC'] == numPoint]
    for indObs, sObs in dfObsListe.iterrows():
        mrk = folium.CircleMarker(location=(sObs.lat, sObs.lon), 
                                  radius=2, color='cyan', fill=True,
                                  popup=folium.Popup('#{} {} {} {} {} (code {})' \
                                                     .format(indObs, sObs.Date, sObs.Horaire,
                                                             sObs.Nombre, sObs['Nom latin'], sObs['Code atlas'])))
        mrk.add_to(mp)
    
    # La position observateur "de mémoire", avec cerles concentriques r=10m + STOC EPS pour l'échelle
    dfTraceListe = dfTraces[dfTraces['Num point ACDC'] == numPoint].copy()
    
    latMem, lonMem = dfTraceListe.iloc[0][['lat_mem', 'lon_mem']]
    mrk = folium.CircleMarker(location=(latMem, lonMem), 
                              radius=6, color='orange', fill=True,
                              popup=folium.Popup('#{} Géoloc. de mémoire'.format(numPoint)))
    mrk.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=10, color='orange', weight=1,
                        popup=folium.Popup('Rayon 10m'))
    crc.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=25, color='orange', weight=1,
                        popup=folium.Popup('Rayon 25m'))
    crc.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=100, color='orange', weight=1,
                        popup=folium.Popup('Rayon 100m'))
    crc.add_to(mp)
    crc = folium.Circle(location=(latMem, lonMem), radius=200, color='orange', weight=1,
                        popup=folium.Popup('Rayon 200m'))
    crc.add_to(mp)
    
    # La trace si disponible
    dfTraceListe.dropna(subset=['ID liste'], inplace=True)

    # a. Les lignes reliant les points
    if len(dfTraceListe) > 1:
        dfTraceListe['obseur_lon_sfd'] = dfTraceListe.obseur_lon.shift(-1)
        dfTraceListe['obseur_lat_sfd'] = dfTraceListe.obseur_lat.shift(-1)

        commListe = dfTraceListe.iloc[0]['Commentaire de la liste']
        compListe = dfTraceListe.iloc[0]['Liste complète ?']
        lines = list(zip(zip(dfTraceListe.obseur_lat, dfTraceListe.obseur_lon),
                         zip(dfTraceListe.obseur_lat_sfd, dfTraceListe.obseur_lon_sfd)))[:-1]
        pline = folium.PolyLine(lines, color='blue', weight=1, opacity=0.6,
                                popup=folium.Popup('#{} {} ({}complète)'.format(numPoint, commListe, '' if compListe else 'in')))
        pline.add_to(mp)

    # b. Les points
    if len(dfTraceListe) > 0:
        for _, sPt in dfTraceListe.iterrows():
            mrk = folium.CircleMarker(location=(sPt.obseur_lat, sPt.obseur_lon), 
                                      popup=folium.Popup('#{}: {}'.format(numPoint, sPt.NumPt)),
                                      radius=5, weight=2, color='red', fill=True)
            mrk.add_to(mp)

mp.fit_bounds(mp.get_bounds())

# Save map as shareable / web-publishable interactive one.
mp.save(f'tmp/ACDC2019b-Points{obserAbbv}-ComparaisonGeolocTraceEtMemoire.html')

# Display map.
mp

# Filtrage et cartographie des données et traces

In [ ]:
fn, obser

In [ ]:
dfObsSel = dfObs.copy()

In [ ]:
dfObsSel['Commentaire de la liste'].unique()

In [ ]:
dfObsSel.Commune.unique()

## 1. Filtrage

### a. Impluvium Volvic 2019 Romain

In [ ]:
obsSelName = 'impluvium'

In [ ]:
# Uniquement les EPOC sur Saint-Ours, Pulvérières, Charbonnières-les-Varennes
dfObsSel.drop(dfObsSel[dfObsSel['Commentaire de la liste'].isnull()].index, inplace=True)
dfObsSel.drop(dfObsSel[~dfObsSel.Commune.isin(['Saint-Ours', 'Pulvérières', 'Charbonnières-les-Varennes'])].index, inplace=True)

dfObsSel.drop(dfObsSel[~dfObsSel['Commentaire de la liste'].str.contains('epoc', case=False)].index, inplace=True)

len(dfObsSel)

In [ ]:
dfObsSel

### b. Forêt de Marcenat

In [ ]:
obsSelName = 'marcenat03'

In [ ]:
# Uniquement les EPOC sur Saint-Didier-la-Forêt, Saint-Rémy-en-Rollat (03)
dfObsSel.drop(dfObsSel[~dfObsSel.Commune.isin(['Saint-Didier-la-Forêt', 'Saint-Rémy-en-Rollat'])].index, inplace=True)

len(dfObsSel)

### c. Planèze

In [ ]:
obsSelName = 'planeze15'

In [ ]:
commPlaneze = ['Cussac', 'Neuvéglise', 'Ternes (Les)', 'Sériers',
               'Tanavelle', 'Valuéjols', 'Paulhac', 'Villedieu', 'Ussel',
               'Coltines', 'Talizat']

In [ ]:
# Uniquement les EPOC sur Saint-Didier-la-Forêt, Saint-Rémy-en-Rollat (03)
dfObsSel.drop(dfObsSel[~dfObsSel.Commune.isin(commPlaneze)].index, inplace=True)

len(dfObsSel)

## 2. Extraction et décodage des traces

In [ ]:
# On peut donc récupérer trace de chaque liste et en extraire les points individuels
formIndCols = ['ID liste', 'Date', 'Liste complète ?', 'Commentaire de la liste']
dfTraces = dfObsSel[formIndCols + ['Trace']].groupby(formIndCols).first()
dfTraces

In [ ]:
len(dfTraces)

In [ ]:
def decoderTrace(trace):
    return [[float(num) for num in xy.strip().split(' ')] for xy in trace[len('LINESTRING('):-len(')')].split(',')]
    
dfTraces.Trace = dfTraces.Trace.apply(decoderTrace)
dfTraces = dfTraces.Trace.apply(pd.Series).stack().reset_index()
dfTraces[['obseur_lon', 'obseur_lat']] = dfTraces.loc[:, 0].apply(pd.Series)
dfTraces.drop(columns=[0], inplace=True)
dfTraces.rename(columns=dict(level_4='NumPt'), inplace=True)
dfTraces

## 3. Cartographie

In [ ]:
dfObsSel.Date.min(), dfObsSel.Date.max()

In [ ]:
# Les formulaires à traiter et leurs nbres de données
dfObsSel[['ID liste', 'Trace']].groupby('ID liste').count().rename(columns=dict(Trace='NbObs'))

In [ ]:
dfObsSel.columns

In [ ]:
# Cartographie des formulaires
# La carte et les couches
mp = folium.Map()
folium.TileLayer(**mdSatArcGis).add_to(mp)

# Le contrôle pour changer de couche
folium.LayerControl().add_to(mp)

# Pour chaque formulaire sélectionné
for idListe in sorted(dfTraces['ID liste'].unique()):
    
    # Les données
    dfObsListe = dfObsSel[dfObsSel['ID liste'] == idListe]
    for indObs, sObs in dfObsListe.iterrows():
        mrk = folium.CircleMarker(location=(sObs['Lat (WGS84)'], sObs['Lon (WGS84)']), 
                                  radius=4, color='cyan', fill=True,
                                  popup=folium.Popup('#{} {} {} {} {} (code {})' \
                                                     .format(indObs, sObs.Date, sObs.Horaire,
                                                             sObs.Nombre, sObs['Nom latin'], sObs['Code atlas'])))
        mrk.add_to(mp)
    
    # La position observateur estimée, avec cerles concentriques r=10m + STOC EPS pour l'échelle
    #latMem, lonMem = dfTraceListe.iloc[0][['lat_mem', 'lon_mem']]
    #mrk = folium.CircleMarker(location=(latMem, lonMem), 
    #                          radius=6, color='orange', fill=True,
    #                          popup=folium.Popup('#{} Géoloc. de mémoire'.format(numPoint)))
    #mrk.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=10, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 10m'))
    #crc.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=25, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 25m'))
    #crc.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=100, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 100m'))
    #crc.add_to(mp)
    #crc = folium.Circle(location=(latMem, lonMem), radius=200, color='orange', weight=1,
    #                    popup=folium.Popup('Rayon 200m'))
    #crc.add_to(mp)
    
    # La trace
    dfTraceListe = dfTraces[dfTraces['ID liste'] == idListe].copy()
    
    # a. Les lignes reliant les points
    if len(dfTraceListe) > 1:
        dfTraceListe['obseur_lon_sfd'] = dfTraceListe.obseur_lon.shift(-1)
        dfTraceListe['obseur_lat_sfd'] = dfTraceListe.obseur_lat.shift(-1)

        commListe = dfTraceListe.iloc[0]['Commentaire de la liste']
        lines = list(zip(zip(dfTraceListe.obseur_lat, dfTraceListe.obseur_lon),
                         zip(dfTraceListe.obseur_lat_sfd, dfTraceListe.obseur_lon_sfd)))[:-1]
        pline = folium.PolyLine(lines, color='blue', weight=2, opacity=0.6,
                                popup=folium.Popup('#{} {}'.format(idListe, commListe)))
        pline.add_to(mp)

    # b. Les points
    if len(dfTraceListe) > 0:
        for _, sPt in dfTraceListe.iterrows():
            mrk = folium.CircleMarker(location=(sPt.obseur_lat, sPt.obseur_lon), 
                                      popup=folium.Popup('#{}: {}'.format(idListe, sPt.NumPt)),
                                      radius=3, weight=2, color='red', fill=True)
            mrk.add_to(mp)

mp.fit_bounds(mp.get_bounds())

# Display map.
mp

In [ ]:
# Save map as shareable / web-publishable interactive one.
mFn = pl.Path(fn).with_suffix(f'.{obsSelName}.html')
mp.save(str(mFn))

HTML(f'<a href="{mFn}" target="_blank">{mFn}</a>')

# Cartographie des communes d'une sélection de données

In [ ]:
# Chargement d'un fichier des sites à peu près à jour
dfSites = pd.read_csv('tmp/SitesFA-20190522.csv', sep='\t', skiprows=1)
dfSites.head()

In [ ]:
dfCommunes.columns

In [ ]:
# Et voici en gros la mairie de chaque commune.
dfCommunes = dfSites[dfSites.Nom == dfSites.Commune.apply(lambda s: s + ' (bourg)')]
len(dfCommunes)

In [ ]:
# Filtrage
dfCommunesSel = dfCommunes[dfCommunes.Commune.isin(dfObsSel.Commune.unique())]

In [ ]:
# Ou pas filtrage
dfCommunesSel = dfCommunes

In [ ]:
# La carte et les couches
mp = folium.Map()
folium.TileLayer(**mdSatArcGis).add_to(mp)

# Le contrôle pour changer de couche
folium.LayerControl().add_to(mp)

# Pour chaque formulaire sélectionné
for _, sCom in dfCommunesSel.iterrows():
    
    # La position observateur estimée, avec cerles concentriques r=10m + STOC EPS pour l'échelle
    mrk = folium.CircleMarker(location=sCom[['Latitude (D.d)', 'Longitude (D.d)']], 
                              radius=10, color='fuchsia', fill=True,
                              popup=folium.Popup('{} - {:02d} ({}m)'.format(sCom.Nom, int(sCom.INSEE)//1000, sCom.Altitude)))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())

# Display map.
mp

# Archives

## Essai extraction traces via format JSON / XML

(avant de savoir qu'elles n'y sont sont pas, le 25/01/2020)

In [ ]:
#src = 'fa'
src = 'ff'

In [ ]:
fn = f'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602{src}.json'

In [ ]:
fn = f'tmp/NaturalistTestTrace-JardinJPM20190523{src}.json'

In [ ]:
fn = 'tmp/NaturalistTestTrace-FormRomainsAutHiv201920fa.json'

In [ ]:
dObsTr = json.load(open(fn))
type(dObsTr), type(dObsTr['data']), dObsTr['data'].keys(), type(dObsTr['data']['forms']), type(dObsTr['data']['sightings'])

In [ ]:
dict(nbFormulaires=len(dObsTr['data']['forms']), nbObsHorsFormulaires=len(dObsTr['data']['sightings']))

In [ ]:
#dObsTr

In [ ]:
#dObsTr['data']['forms'][8]

In [ ]:
for dForm in dObsTr['data']['forms']:
    print(dForm['@id'], ':', dForm['time_start'], dForm['time_stop'], dForm['lat'], dForm['lon'],
                             dForm.get('comment', ''), '=>', len(dForm['sightings']))

In [ ]:
def flattenForm(form):
    flat = odict()
    for k, v in form.items():
        if k == 'sightings':
            continue
        if isinstance(v, dict):
            for sk, sv in v.items():
                if k != 'protocol' or sk == 'protocol_name':
                    flat.update(**{ 'form_'+k+'_'+sk: sv})
        else:
            flat.update(**{ 'form_'+k: v})
    return flat

dfForms = pd.DataFrame(data=[flattenForm(form) for form in dObsTr['data']['forms']])
dfForms.set_index('form_@id', inplace=True)
dfForms

In [ ]:
def flattenSight(sight, formId):
    flat = odict([('form_@id', formId)])
    for k, v in sight.items():
        if isinstance(v, list):
            v = v[0]
        for sk, sv in v.items():
            if isinstance(sv, dict):
                for ssk, ssv in sv.items():
                    flat.update(**{ k+'_'+sk+'_'+ssk: ssv})
            else:
                flat.update(**{ k+'_'+sk: sv})
    return flat

ldfSights = list()
for form in dObsTr['data']['forms']:
    dfSights = pd.DataFrame(data=[flattenSight(sight, form['@id']) for sight in form['sightings']])
    dfSights = dfSights.join(dfForms, on=['form_@id'])
    ldfSights.append(dfSights)
    
dfSights = pd.concat(ldfSights, sort=False, ignore_index=True)
dfSights

In [ ]:
dfSights.columns

In [ ]:
if 'form_protocol_protocol_name' in dfSights.columns:
    dfSights.drop(dfSights[dfSights['form_protocol_protocol_name'] == 'STOC_EPS'].index, inplace=True)
dfSights['form_comment'].fillna('', inplace=True)
dfSights.drop(dfSights[~dfSights['form_comment'].str.contains('ACDC', case=False)].index, inplace=True)

dfSights

In [ ]:
dfSights['date_@ISO8601'].min(), dfSights['date_@ISO8601'].max()

In [ ]:
dfSights[['form_@id']+[col for col in dfSights.columns if col.endswith('lat') or col.endswith('lon')]].head(30)